In [1]:
import importlib.util
import sys

# Step 1: Convert file path to module name
file_path = '/Users/martineberlein/github/EvoGFuzzplusplus/evaluation/resources/mpi/problem_1_GCD/prog_3/buggy.py'
module_name = file_path.replace('/', '.').rstrip('.py')

In [2]:
# Step 2: Load module dynamically
spec = importlib.util.spec_from_file_location(module_name, file_path)
module = importlib.util.module_from_spec(spec)
sys.modules[module_name] = module
spec.loader.exec_module(module)

In [3]:
# Step 3: Access functions or classes
#your_function = getattr(module, 'gcd')
your_class = getattr(module, 'Solution')

# Usage
#your_function()
ob = your_class()

In [4]:
ob.gcd(10, 10)

5

In [5]:
from IPython.display import display, Code
import inspect

# Assume 'your_class' is the dynamically loaded class object
source_code = inspect.getsource(your_class)

# Display with syntax highlighting
display(Code(source_code, language='python'))

class Solution:
    def gcd(self, A, B):
        l=[]
        if A>=B:
            for i in range(1,A):
                if A%i==0 and B%i==0:
                    l.append(i)
            if len(l) <= 1:
                return l[0]
            else:
                return l[-1]
        else:
            for i  in range(1,B):
                 if A%i==0 and B%i==0:
                    l.append(i)
            if len(l) <= 1:
                return l[0]
            else:
                return l[-1]

In [6]:
# Reference implementation using the Euclidean algorithm
class Reference():
    def gcd(self, A, B):
        if A < B:
            A, B = B, A
        while B:
            A, B = B, A % B
        return A

In [7]:
# Comparing the two implementations
print(ob.gcd(10, 10))  # Output should be 10 for correct implementation

ref = Reference()
print(ref.gcd(10, 10))  # Output should be 10 as well

5
10


In [8]:
# Setting up a grammar for fuzzing
from fuzzingbook.Grammars import Grammar
import string 

grammar: Grammar = {
    '<start>': ["<input>"],
    "<input>": ["<first> <second>"],
    "<first>": ["<integer>"],
    "<second>": ["<integer>"],
    "<integer>": ["<onenine><maybe_digits>"],
    "<onenine>": [str(num) for num in range(1, 10)],
    "<digit>": list(string.digits),
    "<maybe_digits>": ["", "<digits>"],
    "<digits>": ["<digit>", "<digit><digits>"],
}

In [9]:
# Constructing an oracle using the reference GCD implementation
from evogfuzz.oracle_construction import construct_oracle, UnexpectedResultError, OracleResult

#error_def = {UnexpectedResultError: OracleResult.BUG}
error_def = {TimeoutError: OracleResult.UNDEF}
oracle = construct_oracle(ob.gcd, ref.gcd, error_def, default_oracle_result=OracleResult.BUG, timeout=1)

In [10]:
# Using Avicenna for diagnosis
from evogfuzz.evogfuzz_class import EvoGGen

initial_inputs = ["10 2", "4 4"]

from evogfuzz import EvoGFuzz

evogfuzz = EvoGFuzz(
    grammar=grammar,
    inputs=initial_inputs,
    oracle=oracle,
    iterations=3,
)

In [11]:
found_exception_inputs = evogfuzz.fuzz()
print(f"EvoGFuzz found {len(found_exception_inputs)} bug-triggering inputs!")

EvoGFuzz found 15 bug-triggering inputs!


In [12]:
for inp in list(found_exception_inputs)[:250]:
    print(str(inp).ljust(30), inp.oracle)

3 3                            BUG
20 20                          BUG
50 50                          BUG
60 60                          BUG
8 8                            BUG
70 70                          BUG
7 7                            BUG
4 4                            BUG
6 6                            BUG
10 10                          BUG
4 4                            BUG
80 80                          BUG
2 2                            BUG
40 40                          BUG
1 1                            BUG


In [13]:
evogfuzz.report

Report for EvoGFuzz
Found 15 failure-inducing inputs (2 Exceptions):
UnexpectedResultError: Results do not match: 14
IndexError: list index out of range: 1

In [14]:
class TestSubject:
    def __init__(self, grammar=None, oracle=None, test_inputs=None):
        self.grammar = grammar or self.default_grammar()
        self.oracle = oracle or self.default_oracle()
        self.test_inputs = test_inputs or self.default_test_inputs()
        
    def default_grammar(self):
        return {}
    
    def default_oracle(self):
        return None
    
    def default_test_inputs(self):
        return []
    
    def to_dict(self):
        return {
            'grammar': self.grammar,
            'oracle': self.oracle,
            'inputs': self.test_inputs,
        }

In [15]:
# Example of subclassing
class GCDTestSubject(TestSubject):
    def default_grammar(self):
        return grammar

    def default_oracle(self):
        error_def = {TimeoutError: OracleResult.UNDEF}
        return construct_oracle(ob.gcd, ref.gcd, error_def, default_oracle_result=OracleResult.BUG)

    def default_test_inputs(self):
        return ["10 2", "4 4"]

In [16]:
subjects = [
    GCDTestSubject
]

for sub in subjects:
    param = sub().to_dict()

In [17]:
param

{'grammar': {'<start>': ['<input>'],
  '<input>': ['<first> <second>'],
  '<first>': ['<integer>'],
  '<second>': ['<integer>'],
  '<integer>': ['<onenine><maybe_digits>'],
  '<onenine>': ['1', '2', '3', '4', '5', '6', '7', '8', '9'],
  '<digit>': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
  '<maybe_digits>': ['', '<digits>'],
  '<digits>': ['<digit>', '<digit><digits>']},
 'oracle': <function evogfuzz.oracle_construction._construct_functional_oracle.<locals>.oracle(inp: evogfuzz.input.Input) -> Tuple[evogfuzz.oracle.OracleResult, Optional[Exception]]>,
 'inputs': ['10 2', '4 4']}

In [18]:
evogfuzz = EvoGFuzz(
    **param, iterations=3,
)
found_exception_inputs = evogfuzz.fuzz()
print(f"EvoGFuzz found {len(found_exception_inputs)} bug-triggering inputs!")

EvoGFuzz found 7 bug-triggering inputs!


In [19]:
for inp in list(found_exception_inputs)[:250]:
    print(str(inp).ljust(30), inp.oracle)

20 20                          BUG
4 4                            BUG
10 10                          BUG
4 4                            BUG
2 2                            BUG
40 40                          BUG
1 1                            BUG


In [20]:
evogfuzz.report

Report for EvoGFuzz
Found 7 failure-inducing inputs (2 Exceptions):
UnexpectedResultError: Results do not match: 6
IndexError: list index out of range: 1

In [21]:
from typing import Union
from pathlib import Path

def load_module_dynamically(path: Union[str, Path], class_name: str, function_name: str):
    # Step 1: Convert file path to module name
    file_path = str(path)
    module_name = file_path.replace('/', '.').rstrip('.py')

    # Step 2: Load module dynamically
    spec = importlib.util.spec_from_file_location(module_name, file_path)
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)
    your_class = getattr(module, class_name)
    function = getattr(your_class(), function_name)
    return function

In [22]:
result = load_module_dynamically('/Users/martineberlein/github/EvoGFuzzplusplus/evaluation/resources/mpi/problem_1_GCD/prog_10/buggy.py', 'Solution', 'gcd')

In [23]:
result(10,10)

10

In [24]:
def get_gcd_subjects(err_def=None, default_oracle=None):
    subjects = []
    for i in range(1,11):
        result = load_module_dynamically(f'/Users/martineberlein/github/EvoGFuzzplusplus/evaluation/resources/mpi/problem_1_GCD/prog_{i}/buggy.py', 'Solution', 'gcd')
        error_def = err_def or {TimeoutError: OracleResult.UNDEF}
        def_oracle = default_oracle or OracleResult.BUG
        oracle = construct_oracle(result, ref.gcd, error_def, default_oracle_result=def_oracle, timeout=0.01)
        subjects.append(GCDTestSubject(oracle=oracle))
    return subjects

In [25]:
get_gcd_subjects()

In [26]:
for sub in get_gcd_subjects():
    print("New Subject: \n\n")
    param = sub.to_dict()
    evogfuzz = EvoGFuzz(
        **param,
        iterations=3,
    )
    found_exception_inputs = evogfuzz.fuzz()
    print(f"EvoGFuzz found {len(found_exception_inputs)} bug-triggering inputs!")
    print(evogfuzz.report)

New Subject: 


EvoGFuzz found 91 bug-triggering inputs!
Report for EvoGFuzz
Found 91 failure-inducing inputs (1 Exceptions):
UnexpectedResultError: Results do not match: 91
New Subject: 


EvoGFuzz found 66 bug-triggering inputs!
Report for EvoGFuzz
Found 66 failure-inducing inputs (1 Exceptions):
UnexpectedResultError: Results do not match: 66
New Subject: 


EvoGFuzz found 5 bug-triggering inputs!
Report for EvoGFuzz
Found 5 failure-inducing inputs (2 Exceptions):
UnexpectedResultError: Results do not match: 4
IndexError: list index out of range: 1
New Subject: 


EvoGFuzz found 0 bug-triggering inputs!
Report for EvoGFuzz
Found 0 failure-inducing inputs (0 Exceptions):

New Subject: 


EvoGFuzz found 2 bug-triggering inputs!
Report for EvoGFuzz
Found 2 failure-inducing inputs (1 Exceptions):
UnexpectedResultError: Results do not match: 2
New Subject: 


EvoGFuzz found 7 bug-triggering inputs!
Report for EvoGFuzz
Found 7 failure-inducing inputs (2 Exceptions):
UnexpectedResultError:

In [27]:
for sub in get_gcd_subjects({TimeoutError: OracleResult.BUG}, OracleResult.UNDEF):
    print("New Subject: \n\n")
    param = sub.to_dict()
    evogfuzz = EvoGFuzz(
        **param,
        iterations=10,
    )
    found_exception_inputs = evogfuzz.fuzz()
    print(f"EvoGFuzz found {len(found_exception_inputs)} bug-triggering inputs!")
    print(evogfuzz.report)

New Subject: 


EvoGFuzz found 0 bug-triggering inputs!
Report for EvoGFuzz
Found 0 failure-inducing inputs (0 Exceptions):

New Subject: 


EvoGFuzz found 236 bug-triggering inputs!
Report for EvoGFuzz
Found 236 failure-inducing inputs (1 Exceptions):
TimeoutError: Function call timed out: 236
New Subject: 


EvoGFuzz found 0 bug-triggering inputs!
Report for EvoGFuzz
Found 0 failure-inducing inputs (0 Exceptions):

New Subject: 


EvoGFuzz found 0 bug-triggering inputs!
Report for EvoGFuzz
Found 0 failure-inducing inputs (0 Exceptions):

New Subject: 


EvoGFuzz found 1 bug-triggering inputs!
Report for EvoGFuzz
Found 1 failure-inducing inputs (1 Exceptions):
TimeoutError: Function call timed out: 1
New Subject: 


EvoGFuzz found 366 bug-triggering inputs!
Report for EvoGFuzz
Found 366 failure-inducing inputs (1 Exceptions):
TimeoutError: Function call timed out: 366
New Subject: 


EvoGFuzz found 31 bug-triggering inputs!
Report for EvoGFuzz
Found 31 failure-inducing inputs (1 Excep